# Data Collection
This noteboook is responsible for autonomously collecting Twitch follower, concurrent view, and total view data.

## Imports

In [1]:
#SQL connection and queries
import MySQLdb as mdb
#Connection failure exiting
import sys
#Getting current time (after data is scraped) for stream table
from time import gmtime, strftime, time, sleep
#Scraping
import urllib2
from bs4 import BeautifulSoup as bs
#Web automation
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## File data
Below the database credentials are read in. These are used for logging into the database, creating the necessary tables (if not already created), and storing data.

In [2]:
db_user = ""
db_pass = ""
db_name = ""
db_host = "localhost"
with open("database_credentials.txt") as f:
    db_user = f.readline().strip()
    db_pass = f.readline().strip()
    db_name = f.readline().strip()

## Database
### Versioning
Below the database is accessed using the specified username, password, and database name above. Note if the below code errors out, the rest of the database operations in this file will not produce expeted results (most likely will flat out not work).

In [3]:
#Connect
#def mdb_get_version():
"""
Connects to the database above using the obtained credentials.
"""
con = None
try:
    con = mdb.connect(host=db_host, user=db_user, passwd=db_pass, db=db_name)
    cur = con.cursor()
    cur.execute("SELECT VERSION()")
    data = cur.fetchone()
    print("Database version: {}".format(data))
except mdb.Error, e:
    print("Error {}: {}".format(e.args[0], e.args[1]))
    sys.exit(1)
finally:
    if con:
        con.close()
#mdb_get_version()

Database version: ('5.7.21-log',)


### Table Creation
The below functions create the necessary tables for data storage if they do not already exist.

In [4]:
#def create_tables():
"""
Creates the necessary tables (if not already created) for Twitch data storage.
"""
con = mdb.connect(host=db_host, user=db_user, passwd=db_pass, db=db_name)

with con:
    cur = con.cursor()

    #Twitch game/creative/irl categories
    sql = """
    CREATE TABLE IF NOT EXISTS categories (
        id INT PRIMARY KEY AUTO_INCREMENT,
        category VARCHAR(255) NOT NULL UNIQUE
    )
    """
    cur.execute(sql)

    #Twitch streamers
    sql = """
    CREATE TABLE IF NOT EXISTS streamers (
        id INT PRIMARY KEY AUTO_INCREMENT,
        streamer VARCHAR(255) NOT NULL UNIQUE
    )
    """
    cur.execute(sql)

    #Individual stream
    """
    It's important to note here that any game, like Overwatch, will be streamed by multiple streamers.
    Similarly, a streamer may stream multiple games, or even stream Overwatch at different times.
    The goal of the learning agents will be to see how streaming variables, like stream time, affect
    concurrent viewership, if at all.
    """
    sql = """
    CREATE TABLE IF NOT EXISTS streams (
        id INT PRIMARY KEY AUTO_INCREMENT,
        current_viewers INT NOT NULL DEFAULT -1,
        created_at DATETIME NOT NULL DEFAULT current_timestamp,
        followers INT NOT NULL DEFAULT -1,
        total_views INT NOT NULL DEFAULT -1,
        streamer_id INT NOT NULL DEFAULT -1,
        FOREIGN KEY (streamer_id)
            REFERENCES streamers(id)
            ON DELETE CASCADE,
        category_id INT NOT NULL DEFAULT -1,
        FOREIGN KEY (category_id)
            REFERENCES categories(id)
            ON DELETE CASCADE
    )
    """
    cur.execute(sql)
#create_tables()

C:\Users\unknown\AppData\Local\conda\conda\envs\twitch_views\lib\site-packages\ipykernel_launcher.py:17: Warning: Table 'categories' already exists
C:\Users\unknown\AppData\Local\conda\conda\envs\twitch_views\lib\site-packages\ipykernel_launcher.py:26: Warning: Table 'streamers' already exists
C:\Users\unknown\AppData\Local\conda\conda\envs\twitch_views\lib\site-packages\ipykernel_launcher.py:52: Warning: Table 'streams' already exists


## Twitch parameters
Below the variables for scraping on Twitch are setup. Currently this includes the following:
* url strings
* categories
    * these were chosen from the top 10 streaming categories on 2/2/2018 at 2:40pm

### URL strings

In [6]:
#String for vieweing all current streams for a specified category
category_string = "https://www.twitch.tv/directory/game/{}"

#String for navigating to a specific user's stream
stream_string = "https://www.twitch.tv/{}"

### Category strings

In [7]:
#Below is a list of the 10 most popular streaming categories on twitch as of
#2/2/2018 @ 2:40pm
safe_chars = "'"
categories = [
    "fortnite",
    "league of legends",
    "playerunknown\'s battlegrounds",
    "grand theft auto v",
    "hearthstone",
    "counter-strike: global offensive",
    "dota 2",
    "irl",
    "world of warcraft",
    "overwatch"
]
#Use reassignment of smaller category list for testing
#categories = ["super mario 64", "paladins", "pokémon red/blue"]
category_map = {}

#Assign category URL mappings (could be avoided by formating URL each time)
print("Categories:")
for i in range(len(categories)):
    c = categories[i]
    category_map[c] = {
        "url": category_string.format(urllib2.quote(c, safe=safe_chars)),
        #Database primary keys are initialized later in this code cell
        "primary_key": -1
    }
    
#Initialize database with categories
try:
    #Connect
    con = mdb.connect(host=db_host, user=db_user, passwd=db_pass, db=db_name)
    #Set autocommit (otherwise inserts won't be propogated to database)
    con.autocommit(True)
    cur = con.cursor(mdb.cursors.DictCursor)
    
    #Only perform following steps if categories table hasn't been initialized
    if 0 == cur.execute("""SELECT * FROM categories"""):
        #Iterate over categories and insert each
        for c in categories:
            #Note in the below replace statement, this is for handling escaping of the
            #single quote in PLAYERUNKNOWN'S BATTLEGROUNDS
            sql = """
            INSERT INTO categories (category) VALUES ('{}')
            """.format(c.replace("'", "''"))
            cur.execute(sql)

    #Get category/id pairs
    sql = """
    SELECT * FROM categories
    """
    cur.execute(sql)
    pairs = cur.fetchall()
    #Assign primary keys
    for p in pairs:
        category_map[p["category"]]["primary_key"] = p["id"]
finally:
    if con:
        con.close()
    
#Print url/id mapping results
for k,v in category_map.iteritems():
    #print("{}, {}, {}".format(k, v["url"], v["primary_key"]))
    print('{}: {{\n\t"url": {},\n\t"primary_key": {}\n}}'.format(k, v["url"], v["primary_key"]))

Categories:
overwatch: {
	"url": https://www.twitch.tv/directory/game/overwatch,
	"primary_key": 10
}
dota 2: {
	"url": https://www.twitch.tv/directory/game/dota%202,
	"primary_key": 7
}
playerunknown's battlegrounds: {
	"url": https://www.twitch.tv/directory/game/playerunknown's%20battlegrounds,
	"primary_key": 3
}
league of legends: {
	"url": https://www.twitch.tv/directory/game/league%20of%20legends,
	"primary_key": 2
}
grand theft auto v: {
	"url": https://www.twitch.tv/directory/game/grand%20theft%20auto%20v,
	"primary_key": 4
}
irl: {
	"url": https://www.twitch.tv/directory/game/irl,
	"primary_key": 8
}
fortnite: {
	"url": https://www.twitch.tv/directory/game/fortnite,
	"primary_key": 1
}
hearthstone: {
	"url": https://www.twitch.tv/directory/game/hearthstone,
	"primary_key": 5
}
world of warcraft: {
	"url": https://www.twitch.tv/directory/game/world%20of%20warcraft,
	"primary_key": 9
}
counter-strike: global offensive: {
	"url": https://www.twitch.tv/directory/game/counter-strik

## Data collection

### Converting inputs
The below function converts strings of the form "1,234,567,890" to numerics 1234567890.

In [8]:
def twitch_numberfy(s):
    return int(s.replace(",", ""))

### Functional output separation
The below code just prints out visual separation when the loop is performing different tasks. Separation is used to divide output by category, scraping, and writing.

In [9]:
def output_separation(phrase, sep_char, border_height, border_width):
    """
    This funciton provides the following functionality:
        output_separation("hi", "|", 3, 10) yields
        ||||||||||
        ||||||||||
        ||||||||||
        ||||hi||||
        ||||||||||
        ||||||||||
        ||||||||||
    Note that sep_char must be a single character.
    """
    for i in range(border_height):
        print(sep_char*border_width)
    print(phrase.center(border_width, sep_char))
    for i in range(border_height):
        print(sep_char*border_width)

### Scraping
The scraping module below is responsible for collecting data about all live streams currently streaming under 1 of the categories defined above. The driver does the following steps:
* Navigate to category *C*'s browse page
    * Collect the available streams (these should be the streams with the most concurrent viewers)
    * Scroll to the bottom of the page in an attempt to load more streams
    * Repeat this process until no new streams are found
* Store category *C* streams

In [10]:
first_time = time()

In [11]:
#Spin up browser
driver = webdriver.Chrome()

#Number of times to scroll down a page at a time
scrolls = 25

#Time to sleep between scrolls
sleeptime = 0.5

#Used for output formatting
out_width = 72

#Iterate over each category
for c in category_map:
    output_separation(c, " ", 8, out_width)
    start = time()
    #Load up page
    driver.get(category_map[c]["url"])
    
    #Streams for storage
    streams = []
    stream_map = {}
    
    try:
        #Wait for page to load at least 1 stream
        element = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CLASS_NAME, "stream-thumbnail")))
        
        #Loop to get more streams
        output_separation("COLLECTING STREAM EVENTS", "|", 4, out_width)
        prev_stream_count = -1
        while True:            
            #Assign global scrollable element for access through other scripts
            scroll_script = """
            window.s = document.getElementsByClassName("root-scrollable__wrapper")[0].parentElement.parentElement;
            """
            driver.execute_script(scroll_script)
            
            #Scroll
            scroll_script = """
            window.s.scrollTo(0, window.s.scrollHeight);
            """
            output_separation("SCROLLING FOR {}s".format(scrolls * sleeptime), "#", 0, out_width)
            for i in range(scrolls):
                driver.execute_script(scroll_script)
                sleep(sleeptime)
                
            #Find streams
            streams = driver.find_elements_by_class_name("live-channel-card__videos")
            #Find text in parent containing concurrent viewers
            streams = map(lambda e: e.find_element_by_xpath("..").text, streams)
            
            #If last stream obtained has 0 viewers, break
            if prev_stream_count == len(streams):
                break
            prev_stream_count = len(streams)
    finally:
        output_separation("RETRIEVAL AND UNWRAPPING: {}s".format(time() - start), "~", 1, out_width)
    
    #Iterate over streams
    output_separation("SCRAPING STREAM DATA", "-", 4, out_width)
    for s in streams:
        #Add kvp mapping for streamer
        streamer = s.split(" ")[-1]
        #Foreign players with non-english characters have "(name_here)"
        if "(" in streamer:
            streamer = streamer[1:-1]
        
        #Load up page
        print("{}".ljust(40 - len(streamer), " ").format(streamer)),
        driver.get(stream_string.format(streamer))
        try:
            #Wait for page to load total viewer count
            #element = WebDriverWait(driver, 5).until(
            #        EC.text_to_be_present_in_element((By.CLASS_NAME, "tw-stat__value")))
            sleep(1)
            
            #Record stats
            #Followers
            top_stats = driver.find_elements_by_class_name("channel-header__item-count")
            if len(top_stats) == 0:
                #Channel lookup error or banned (banned unlikely)
                print("[BANNED->SKIPPED]")
                continue
            followers = twitch_numberfy(
                top_stats[1].find_element_by_tag_name("span").get_attribute("innerHTML"))
            #Current viewers and total views
            bottom_stats = driver.find_elements_by_class_name("tw-stat__value")
            if len(bottom_stats) == 0:
                #Hosting, so skip
                print("[HOSTING->SKIPPED]")
                continue
            elif len(bottom_stats) == 1:
                #Offline, so skip
                print("[OFFLINE->SKIPPED]")
                continue
            current_viewers = twitch_numberfy(bottom_stats[0].get_attribute("innerHTML"))
            total_views = twitch_numberfy(bottom_stats[1].get_attribute("innerHTML"))
            
            #Store in dictionary
            stream_map[streamer] = {
                "followers": followers,
                "current_viewers": current_viewers,
                "total_views": total_views
            }
            print("[DONE]")
        except (RuntimeError, TypeError, NameError):
            print("[ERROR->SKIPPED]")
            
    #Write streams and streamers to database
    output_separation("STORING EVENTS AND STREAMERS", "*", 4, out_width)
    try:
        #Connect
        con = mdb.connect(host=db_host, user=db_user, passwd=db_pass, db=db_name)
        #Set autocommit (otherwise inserts won't be propogated to database)
        con.autocommit(True)
        cur = con.cursor(mdb.cursors.DictCursor)
        
        #Iterate over streams for this category
        category_id = category_map[c]["primary_key"]
        for k,v in stream_map.iteritems():
            #k is streamer (twitch name)
            #v is dictionary holding followers, current_viewers, and total_views
            
            #Get streamer ids. If the streamer is undiscovered/new, create
            #a database entry first
            sql = """
            SELECT * FROM streamers WHERE streamer='{}'
            """.format(k)
            result = cur.execute(sql)
            #print("Result: {}".format(result))

            streamer_id = -1
            #If 0 < result, then streamer already has an entry, so get their primary key
            if 0 < result:
                #Only 1 can exist, the varchar is unique
                streamer_id = cur.fetchall()[0]["id"]
            #Otherwise, this is a new streamer
            else:
                #Insert new entry
                sql = """
                INSERT INTO streamers (streamer) VALUES ('{}')
                """.format(k)
                cur.execute(sql)
                #Get primary key of last row inserted into ANY table
                streamer_id = cur.lastrowid
            
            #Streamer information has been obtained (and added if necessary).
            #Create stream event entry
            sql = """
            INSERT INTO streams
                (current_viewers, followers, total_views, streamer_id, category_id)
                VALUES
                ({}, {}, {}, {}, {})
            """.format(
                v["current_viewers"],
                v["followers"],
                v["total_views"],
                streamer_id,
                category_id)
            cur.execute(sql)
            
            #Output storage
            print("{}".ljust(40 - len(k), " ").format(k)),
            print("[OKAY]")
    finally:
        if con:
            con.close()
#Close browser
driver.close()

                                                                        
                                                                        
                                                                        
                                                                        
                                                                        
                                                                        
                                                                        
                                                                        
                               overwatch                                
                                                                        
                                                                        
                                                                        
                                                                        
                                                   

xSpyInPinkx                            [DONE]
iAmThePhoeniix                         [DONE]
Vyolent                                [DONE]
BetterGamingLive                       [DONE]
gaynyan_tv                             [DONE]
RCW05                                  [DONE]
Izaylia                                [OFFLINE->SKIPPED]
peachymess                             [DONE]
wompmacho                              [DONE]
KingSlev                               [DONE]
LGNDRY91                               [DONE]
Dinodude                               [DONE]
ShinHito                               [DONE]
Ksenia_Noche                           [DONE]
xxalarakxx                             [DONE]
Judyeee                                [DONE]
KnX92i                                 [DONE]
misterbayman                           [DONE]
KDoubleU_                              [DONE]
Cobowled                               [DONE]
shapes_tv                              [DONE]
olga_bulochka         

Patu94                                 [DONE]
TheBlueBomber                          [DONE]
ementalo                               [DONE]
bbygrlpls                              [DONE]
Heartless_s5                           [DONE]
DoxyCris                               [DONE]
xhellokayla                            [DONE]
GearSecondqt                           [DONE]
SweetWishes                            [DONE]
minwoo_ow                              [DONE]
Monguer_Horse                          [DONE]
PuddingLPTV                            [DONE]
FrankieMG1                             [DONE]
ChoccymilkOW                           [DONE]
s7tek                                  [DONE]
DuckStylezz                            [OFFLINE->SKIPPED]
Abnormous_Prime                        [DONE]
yoqern                                 [DONE]
Pescaetee                              [DONE]
AnneDro                                [DONE]
VGcafe                                 [DONE]
CalabozosYMurlocs     

wake5                                  [DONE]
LazyFlavorA                            [DONE]
roye                                   [DONE]
OnceMoreISaid                          [DONE]
RPGMookie                              [DONE]
FalcettiTime                           [DONE]
SPACEMAN_OW                            [DONE]
GringoSoBulgy                          [DONE]
NetworkGangstaa                        [DONE]
zombs_ow                               [HOSTING->SKIPPED]
Aqede_PL                               [DONE]
ChezNoNo                               [DONE]
RoGeR_OW                               [DONE]
mummydust_                             [DONE]
Commanderphilly                        [OFFLINE->SKIPPED]
quellaa                                [OFFLINE->SKIPPED]
Minazorush                             [DONE]
chaes37                                [DONE]
Khry_OW                                [OFFLINE->SKIPPED]
hexm0rtis                              [DONE]
PlatanoSunshine                 

theidark12                             [DONE]
OverNineTeam                           [DONE]
Omi_Wan_Kenobi                         [DONE]
Dalton_OW                              [DONE]
Arikie                                 [DONE]
HoMaLoN                                [DONE]
keinshinxx                             [DONE]
FpsTwitcher1                           [DONE]
Autopoietico                           [DONE]
syncupp                                [DONE]
guibiskup                              [DONE]
Mhelkor                                [OFFLINE->SKIPPED]
mmegaming01                            [OFFLINE->SKIPPED]
microwolff                             [HOSTING->SKIPPED]
Snakered_                              [DONE]
RNF1234                                [DONE]
erraticpixels                          [DONE]
Anhur4                                 [DONE]
SlaaKhor                               [OFFLINE->SKIPPED]
xThierryx                              [DONE]
htroger                         

...

spapqp                                 [DONE]
tksebastian                            [DONE]
SquishedBro                            [DONE]
ry49820000                             [DONE]
SpaceTimeTaco                          [DONE]
TheEnesterShow                         [DONE]
ogclickbaitnate                        [DONE]
Vampire_Metalhead                      [DONE]
inocentk1                              [DONE]
SeptaBusOrgy                           [DONE]
jbpicudo                               [DONE]
hwagananda                             [DONE]
Pimmelstuffe                           [DONE]
fuzeexe                                [DONE]
fellkachina                            [DONE]
djv4145                                [DONE]
html10910                              [DONE]
bygeveze88                             [OFFLINE->SKIPPED]
unknowncracka                          [DONE]
parkitv                                [DONE]
YoruyenYoruk                           [DONE]
cuteluv22             

movingcolors                           [OKAY]
caitlinkim                             [OKAY]
MissSynchronicity                      [OKAY]
musictwitch                            [OKAY]
MrFrush24                              [OKAY]
Slurpwis                               [OKAY]
KaryssaKilljoys                        [OKAY]
TipsyKait                              [OKAY]
rawtopics                              [OKAY]
schlim6                                [OKAY]
Pimmelstuffe                           [OKAY]
CaptainContent_                        [OKAY]
k4iley                                 [OKAY]
HighOf_420                             [OKAY]
GoonerGooch                            [OKAY]
CiraCorellia                           [OKAY]
deception95                            [OKAY]
cololc                                 [OKAY]
BaerVerteidiger                        [OKAY]
DevilCloudGaming                       [OKAY]
yukastar3                              [OKAY]
zoelldor                          

JFlexxx                                [OKAY]
writerAmy                              [OKAY]
pobeditelb_nyan_loto                   [OKAY]
flanerietoast                          [OKAY]
wilrus                                 [OKAY]
Awptimustak                            [OKAY]
PROMEII                                [OKAY]
TheGAas                                [OKAY]
fellkachina                            [OKAY]
PatoPapao                              [OKAY]
shiano0099                             [OKAY]
JBurr23                                [OKAY]
OMGDISASTERJAYHD                       [OKAY]
chick3nandf1sh                         [OKAY]
x457457                                [OKAY]
ookami_kun                             [OKAY]
HomebodyLifestyle                      [OKAY]
TheSophie_                             [OKAY]
tavo_user                              [OKAY]
Piccolino9                             [OKAY]
dragonmotherk                          [OKAY]
inocentk1                         

||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
||||||||||||||||||||||||COLLECTING STREAM EVENTS||||||||||||||||||||||||
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
##########################SCROLLING FOR 12.5s###########################
##########################SCROLLING FOR 12.5s###########################
##########################SCROLLING FOR 12.5s###########################
##########################SCROLLING FOR 12.5s###########################
##########################SCROLLING FOR 12.5s######

WebDriverException: Message: chrome not reachable
  (Session info: chrome=64.0.3282.186)
  (Driver info: chromedriver=2.35.528161 (5b82f2d2aae0ca24b877009200ced9065a772e73),platform=Windows NT 10.0.16299 x86_64)


In [12]:
print("total time: {}s".format(time() - first_time))

total time: 16909.4720001s
